Data:

In this notebook data is downloaded via Kaggle API. 
If you do not want to use Kaggle API like it is done in this notebook, here is manual download option

https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus

In [ ]:
!pip install -q -U trax

In [ ]:
import os 
import numpy as np
import pandas as pd
import random as rnd

import trax 
from trax.supervised import training
from trax import layers as tl

# set random seeds to make this notebook easier to replicate
rnd.seed(33)

data handling

In [ ]:
! pip install -q kaggle                                         # directdownload from kaggle but first upload Apikey in files
! mkdir ~/.kaggle                                               # https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/ , https://www.kaggle.com/general/74235 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle competitions download <name-of-competition>
! kaggle datasets download -d abhinavwalia95/entity-annotated-corpus  
! unzip entity-annotated-corpus.zip

In [ ]:
import pandas as pd
data = pd.read_csv('./ner_dataset.csv', encoding= 'unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [ ]:
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok


token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data.head()

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,Thousands,NNS,O,22293,12
1,NaN,of,IN,O,25282,12
2,NaN,demonstrators,NNS,O,8469,12
3,NaN,have,VBP,O,28724,12
4,NaN,marched,VBN,O,17930,12


In [ ]:
data_fillna = data.fillna(method='ffill', axis=0)
# Groupby and collect columns
data_group = data_fillna.groupby(
['Sentence #'],as_index=False
)[['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx']].agg(lambda x: list(x))
# Visualise data
data_group.head()

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[22293, 25282, 8469, 28724, 17930, 14096, 2990...","[12, 12, 12, 12, 12, 12, 2, 12, 12, 12, 12, 12..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[6145, 15474, 32481, 34880, 5341, 16135, 183, ...","[7, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[4432, 16357, 4380, 5648, 9914, 25603, 15224, ...","[12, 12, 16, 12, 12, 12, 12, 12, 2, 12, 12, 12..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","[7157, 26339, 4205, 8401, 15107, 3355, 237, 18...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]"
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...","[19262, 7815, 26412, 18918, 24866, 32010, 1819...","[2, 12, 12, 15, 3, 12, 16, 12, 2, 12, 7, 12, 7..."


In [ ]:
vocab, tag_map = token2idx, tag2idx
## added pad token with vocabulary size value and 'UNK' for unknown words
vocab['<PAD>'] = len(vocab)
vocab['UNK'] = len(vocab) + 17
## replaced only 0th and 12th mappings, so that 'O' meaning "other" or "out of entities" corresponds to 0 not 12
tag_map = {'O': 0, 'I-per': 1, 'B-tim': 2, 'I-geo': 3, 'I-art': 4, 'I-gpe': 5, 'B-gpe': 6, 'B-art': 7, 'I-org': 8, 'B-geo': 9, 'I-nat': 10, 'B-eve': 11, 'I-eve': 12, 'I-tim': 13, 'B-nat': 14, 'B-org': 15, 'B-per': 16}

print(tag2idx)
vocab['<PAD>'], type(vocab)

In [ ]:
# from sklearn.model_selection import train_test_split
# from keras.utils import to_categorical, pad_sequences


# def get_pad_train_test_val(data_group, data):

#     #get max token and tag length
#     n_token = len(list(set(data['Word'].to_list())))
#     n_tag = len(list(set(data['Tag'].to_list())))

#     #Pad tokens (X var)    
#     tokens = data_group['Word_idx'].tolist()
#     maxlen = max([len(s) for s in tokens])
#     pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

#     #Pad Tags (y var) and convert it into one hot encoding
#     tags = data_group['Tag_idx'].tolist()
#     pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
#     n_tags = len(tag2idx)
#     pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
#     #Split train, test and validation set
#     tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
#     train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

#     print(
#         'train_tokens length:', len(train_tokens),
#         '\ntrain_tags length:', len(train_tags),
#         '\nval_tokens:', len(val_tokens),
#         '\nval_tags:', len(val_tags),
#         '\ntest_tokens length:', len(test_tokens),
#         '\ntest_tags:', len(test_tags),
#     )
    
#     return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

# train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)


def get_params(vocab, tag_map):
    sentences = []
    labels = []

    for i in range(len(data_group.iloc[:])):
        # replace each token by its index if it is in vocab

        s = [ vocab[token] for token in data_group.iloc[i]['Word'] ]
        sentences.append(s)

        # replace each label by its index
        l = [ tag_map[label] for label in data_group.iloc[i]['Tag'] ] # I added plus 1 here
        labels.append(l) 

    return sentences, labels, len(sentences)


t_sentences, t_labels, t_size = get_params(vocab, tag_map)
v_sentences, v_labels, v_size = get_params(vocab, tag_map)
test_sentences, test_labels, test_size = get_params(vocab, tag_map)

del(data) # remove to free memory
del(data_group) # remove to free memory

In [ ]:
# def get_vocab(vocab_path, tags_path):
#     vocab = {}
#     with open(vocab_path) as f:
#         for i, l in enumerate(f.read().splitlines()):
#             vocab[l] = i  # to avoid the 0
#         # loading tags (we require this to map tags to their indices)
#     vocab['<PAD>'] = len(vocab) # 35180
#     tag_map = {}
#     with open(tags_path) as f:
#         for i, t in enumerate(f.read().splitlines()):
#             tag_map[t] = i 
    
#     return vocab, tag_map

# def get_params(vocab, tag_map, sentences_file, labels_file):
#     sentences = []
#     labels = []

#     with open(sentences_file) as f:
#         for sentence in f.read().splitlines():
#             # replace each token by its index if it is in vocab
#             # else use index of UNK_WORD
#             s = [vocab[token] if token in vocab 
#                  else vocab['UNK']
#                  for token in sentence.split(' ')]
#             sentences.append(s)

#     with open(labels_file) as f:
#         for sentence in f.read().splitlines():
#             # replace each label by its index
#             l = [tag_map[label] for label in sentence.split(' ')] # I added plus 1 here
#             labels.append(l) 
#     return sentences, labels, len(sentences)


# vocab, tag_map = get_vocab('./words.txt', './tags.txt')
# t_sentences, t_labels, t_size = get_params(vocab, tag_map, './train_sentences.txt', './train_labels.txt')
# v_sentences, v_labels, v_size = get_params(vocab, tag_map, './val_sentences.txt', './val_labels.txt')
# test_sentences, test_labels, test_size = get_params(vocab, tag_map, './test_sentences.txt', './test_labels.txt')

In [ ]:
# vocab translates from a word to a unique number
print('vocab["the"]:', vocab["the"])
# Pad token
print('padded token:', vocab['<PAD>'])

In [ ]:
print(tag_map)

In [ ]:
# Exploring information about the data
print('The number of outputs is tag_map', len(tag_map))
# The number of vocabulary tokens (including <PAD>)
g_vocab_size = len(vocab)
print(f"Num of vocabulary words: {g_vocab_size}")
print('The training size is', t_size)
print('The validation size is', v_size)
print('An example of the first sentence is', t_sentences[0])
print('An example of its corresponding label is', t_labels[0])

In [ ]:
def data_generator(batch_size, x, y, pad, shuffle=False, verbose=False):
    '''
      Input: 
        batch_size - integer describing the batch size
        x - list containing sentences where words are represented as integers
        y - list containing tags associated with the sentences
        shuffle - Shuffle the data order
        pad - an integer representing a pad character
        verbose - Print information during runtime
      Output:
        a tuple containing 2 elements:
        X - np.ndarray of dim (batch_size, max_len) of padded sentences
        Y - np.ndarray of dim (batch_size, max_len) of tags associated with the sentences in X
    '''
    
    # count the number of lines in data_lines
    num_lines = len(x)
    
    # create an array with the indexes of data_lines that can be shuffled
    lines_index = [*range(num_lines)]
    
    # shuffle the indexes if shuffle is set to True
    if shuffle:
        rnd.shuffle(lines_index)
    
    index = 0 # tracks current location in x, y
    while True:
        buffer_x = [0] * batch_size # Temporal array to store the raw x data for this batch
        buffer_y = [0] * batch_size # Temporal array to store the raw y data for this batch
                
        # Copy into the temporal buffers the sentences in x[index : index + batch_size] 
        # along with their corresponding labels y[index : index + batch_size]
        # Find maximum length of sentences in x[index : index + batch_size] for this batch. 
        # Reset the index if we reach the end of the data set, and shuffle the indexes if needed.
        max_len = 0
        for i in range(batch_size):
             # if the index is greater than or equal to the number of lines in x
            if index >= num_lines:
                # then reset the index to 0
                index = 0
                # re-shuffle the indexes if shuffle is set to True
                if shuffle:
                    rnd.shuffle(lines_index)
            
            # The current position is obtained using `lines_index[index]`
            # Store the x value at the current position into the buffer_x
            buffer_x[i] = x[lines_index[index]]
            
            # Store the y value at the current position into the buffer_y
            buffer_y[i] = y[lines_index[index]]
            
            lenx = len(x[lines_index[index]])    #length of current x[]
            if lenx > max_len:
                max_len = lenx                   #max_len tracks longest x[]
            
            # increment index by one
            index += 1


        # create X,Y, NumPy arrays of size (batch_size, max_len) 'full' of pad value
        X = np.full((batch_size, max_len), pad)
        Y = np.full((batch_size, max_len), pad)

        # copy values from lists to NumPy arrays. Use the buffered values
        for i in range(batch_size):
            # get the example (sentence as a tensor)
            # in `buffer_x` at the `i` index
            x_i = buffer_x[i]
            
            # similarly, get the example's labels
            # in `buffer_y` at the `i` index
            y_i = buffer_y[i]
            
            # Walk through each word in x_i
            for j in range(len(x_i)):
                # store the word in x_i at position j into X
                X[i, j] = x_i[j]
                
                # store the label in y_i at position j into Y
                Y[i, j] = y_i[j]

                
        if verbose: print("index=", index)
        yield((X,Y))

In [ ]:
batch_size = 5
mini_sentences = t_sentences[0: 8]
mini_labels = t_labels[0: 8]
dg = data_generator(batch_size, mini_sentences, mini_labels, vocab["<PAD>"], shuffle=False, verbose=True)
X1, Y1 = next(dg)
X2, Y2 = next(dg)
print(Y1.shape, X1.shape, Y2.shape, X2.shape)
print(X1[0][:], "\n", Y1[0][:])

In [ ]:
def NER(tags, vocab_size=35181, d_model=50):
    '''
      Input: 
        tag_map - dictionary that maps the tags to numbers
        vocab_size - integer containing the size of the vocabulary
        d_model - integer describing the embedding size
      Output:
        model - a trax serial model
    '''
    model = tl.Serial(
      tl.Embedding(vocab_size, d_model), # Embedding layer
      tl.LSTM(d_model), # LSTM layer
      tl.Dense(len(tags)), # Dense layer with len(tags) units
      tl.LogSoftmax()  # LogSoftmax layer
      )
    
    return model

In [ ]:
model = NER(tag_map)
# display your model
print(model)

In [ ]:
rnd.seed(33)

batch_size = 64

# Create training data, mask pad id=35180 for training.
train_generator = trax.data.inputs.add_loss_weights(
    data_generator(batch_size, t_sentences, t_labels, vocab['<PAD>'], True),
    id_to_mask=vocab['<PAD>'])

# Create validation data, mask pad id=35180 for training.
eval_generator = trax.data.inputs.add_loss_weights(
    data_generator(batch_size, v_sentences, v_labels, vocab['<PAD>'], True),
    id_to_mask=vocab['<PAD>'])

In [ ]:
def train_model(NER, train_generator, eval_generator, train_steps=1, output_dir='model'):
    '''
    Input: 
        NER - the model you are building
        train_generator - The data generator for training examples
        eval_generator - The data generator for validation examples,
        train_steps - number of training steps
        output_dir - folder to save your model
    Output:
        training_loop - a trax supervised training Loop
    '''
    train_task = training.TrainTask(
      train_generator,  # A train data generator
      loss_layer = tl.CrossEntropyLoss(), # A cross-entropy loss function
      optimizer = trax.optimizers.Adam(0.01), # The adam optimizer
    )

    eval_task = training.EvalTask(
      labeled_data = eval_generator,  # A labeled data generator
      metrics = [tl.CrossEntropyLoss(), tl.Accuracy()], # Evaluate with cross-entropy loss and accuracy
      n_eval_batches = 10 # Number of batches to use on each evaluation
    )

    training_loop = training.Loop(
        NER, # A model to train
        train_task, # A train task
        eval_tasks = [eval_task], # The evaluation task
        output_dir = output_dir) # The output directory

    # Train with train_steps
    training_loop.run(n_steps = train_steps)

    return training_loop

In [ ]:
train_steps = 100            # In coursera we can only train 100 steps
!rm -f 'model/model.pkl.gz'  # Remove old model.pkl if it exists

# Train the model
training_loop = train_model(NER(tag_map), train_generator, eval_generator, train_steps)

In [ ]:
model = NER(tag_map)
model.init(trax.shapes.ShapeDtype((1, 1), dtype=np.int32))

model.init_from_file('./model/model.pkl.gz', weights_only=True)

In [ ]:
a = np.array([1, 2, 3, 4])
a == 2

In [ ]:
x, y = next(data_generator(len(test_sentences), test_sentences, test_labels, vocab['<PAD>']))
print("input shapes", x.shape, y.shape)

In [ ]:
tmp_pred = model(x)
print(type(tmp_pred))
print(f"tmp_pred has shape: {tmp_pred.shape}")

In [ ]:
def evaluate_prediction(pred, labels, pad):
    """
    Inputs:
        pred: prediction array with shape 
            (num examples, max sentence length in batch, num of classes)
        labels: array of size (batch_size, seq_len)
        pad: integer representing pad character
    Outputs:
        accuracy: float
    """
    outputs = np.argmax(pred, axis=2)
    print("outputs shape:", outputs.shape)

    mask = labels != pad
    print("mask shape:", mask.shape, "mask[0][20:30]:", mask[0][20:30])

    accuracy = np.sum(outputs == labels) / float(np.sum(mask))

    return accuracy


In [ ]:
accuracy = evaluate_prediction(model(x), y, vocab['<PAD>'])
print("accuracy: ", accuracy)

In [ ]:
def predict(sentence, model, vocab, tag_map):
    s = [vocab[token] if token in vocab else vocab['UNK'] for token in sentence.split(' ')]
    batch_data = np.ones((1, len(s)))
    batch_data[0][:] = s
    sentence = np.array(batch_data).astype(int)
    output = model(sentence)
    outputs = np.argmax(output, axis=2)
    labels = list(tag_map.keys())
    pred = []
    for i in range(len(outputs[0])):
        idx = outputs[0][i] 
        pred_label = labels[idx]
        pred.append(pred_label)
    return pred

In [ ]:
# Try the output for the introduction example
#sentence = "Many French citizens are goin to visit Morocco for summer"
#sentence = "Sharon Floyd flew to Miami last Friday"
#s = [vocab[token] if token in vocab else vocab['UNK'] for token in sentence.split(' ')]

sentence = "Peter Navarro, the White House director of trade and manufacturing policy of U.S, said in an interview on Sunday morning that the White House was working to prepare for the possibility of a second wave of the coronavirus in the fall, though he said it wouldn’t necessarily come"

predictions = predict(sentence, model, vocab, tag_map)
for x,y in zip(sentence.split(' '), predictions):
    if y != 'O':
        print(x,y)